In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
import os.path
import sys
import array
import time
import serial

import matplotlib.pyplot as plt
import numpy as np

def setVol(ser,n):
    """
    input: 
        n:the modulation voltage in Volte
    output: 
        the modulation voltage after re-setting
    set the modulation voltage to n and return voltage at present
    """
    st='SLVL'+str(n)+'\n'
    ser.write(str.encode(st))
    ser.write(str.encode('SLVL?\n')) #return the modulation voltage after re-setting
    s = ser.read(256);
    return s

def setSens(ser,n):
    """
    input: 
        n:the modulation voltage in Volte
    output: 
        the modulation voltage after re-setting
    set the modulation voltage to n and return voltage at present
    """
    st='SENS'+str(n)+'\n'
    ser.write(str.encode(st))
    ser.write(str.encode('SLVL?\n')) #return the modulation voltage after re-setting
    s = ser.read(256);
    return s

def cmdser(ser,cmd):
    """
    input:
        ser: your serial
        cmd: strings of command
            '*IDN?'= Queries the device identification
            'OUTP?1' = Queries the value of X (CH1)
            for other command, refer: http://users.df.uba.ar/dgrosz/material%20adicional/manual%20lock-in%20SR830.pdf from page 85.
    output:
        natural return
    giving command to SR830
    """
#     print('Sending:', cmd)
    ser.write(str.encode(cmd+'\n'))#read output in channel 1
    s = ser.read(256);
    if len(s) > 0:
        print(s)
    return(s)
def cmd(ser,cmd):
    ser.write(str.encode(cmd+'\n'))#read output in channel 1
    s = ser.read(256);
    return(s)

In [9]:
def timeCons(s,num,unit):
    d1 = ['1','3','10','30','100','300']
    d2 = ['us','ms','s','ks']
    x = d1.index(num)
    y = d2.index(unit)
    index = x+y*6-2
    if(index<0 or index>19):
        print('invalid input')
    else:
        cmd(s,'OFLT'+str(index))
        
def sens(s,num,unit):
    d1 = ['1','2','5','10','20','50','100','200','500']
    d2 = ['nV/fA','uV/pA','mV/nA','V/uA']
    x = d1.index(num)
    y = d2.index(unit)
    index =x+y*9-1
    if(index<0 or index>26):
        print('invalid input')
    else:
        cmd(s,'OFLT'+str(index))
#         return(index)

In [4]:
def Volbar(s):
    interact(setVol,ser=fixed(s),Vol=widgets.IntSlider(min=-30, max=30, step=1, value=0))

def plotOut(ser,timeout=0.3):
    o1=[]
    o2=[]
    while(True):
        a1 = cmdser(ser,'OUTP?1')
        a2 = cmdser(ser,'OUTP?2')
        a1.decode();
        a2.decode();
        b1 = float(a1[0:4]);
        b2 = float(a2[0:4]);
        
        o1.append(b1);
        o2.append(b2);

        # Limit o1 and o2 lists to 30 items
        o1=o1[-30:]
        o2=o2[-30:]
        
        plt.cla();
        plt.plot(o1,lable='Out1');
        plt.plot(o2,lable='Out2');
        plt.legend()
        time.sleep(timeout)
        plt.show()

def readOut1(ser,timeout=0.3):
    while True:
        a1 = cmdser(ser,'OUTP?1');
        print("\r"+"Out1="+o1,end="");
        time.sleep(timeout)

def readOut2(ser,timeout=0.3):
    while True:
        a2 = cmdser(ser,'OUTP?2');
        print("\r"+"Out2="+o1,end="");
        time.sleep(timeout)

In [5]:
from multiprocessing import Process
def runInParallel(*fns):
  proc = []
  for fn in fns:
    p = Process(target=fn)
    p.start()
    proc.append(p)
  for p in proc:
    p.join()

In [6]:
def guiAll(Port,timeout=0.3):
    ser = serial.Serial(Port, 9600, timeout=1)
    t = timeout;
    runInParallel(Volbar(ser),readOut1(ser,t),readOut2(ser,t),plotOut(ser,t))

In [7]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
for p in ports:
    print(p.device)
print(len(ports), 'ports found')

/dev/cu.Bluetooth-Incoming-Port
/dev/cu.ZebraMachine-SPPDev
2 ports found


# testing

In [6]:
import serial, sys
port = '/dev/cu.usbserial-PX4IHXWN'
baudrate = 9600
ser = serial.Serial(port,baudrate,timeout=1)

In [ ]:
#setVol(ser,n)

In [ ]:
interact(timeCons,s=fixed(ser),num=['1','3','10','30','100','300'],unit=['us','ms','s','ks'])
interact(sens,s=fixed(ser),num=['1','2','5','10','20','50','100','200','500'],unit=['nV/fA','uV/pA','mV/nA','V/uA'])

plot test

In [10]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import numpy as np
import random
import serial

#initialize serial port
ser = serial.Serial()
ser.port = '/dev/ttyACM0' #Arduino serial port
ser.baudrate = 9600
ser.timeout = 10 #specify timeout when using readline()
ser.open()
if ser.is_open==True:
	print("\nAll right, serial port now open. Configuration:\n")
	print(ser, "\n") #print serial parameters

# Create figure for plotting
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
xs = [] #store trials here (n)
ys = [] #store relative frequency here
rs = [] #for theoretical probability

# This function is called periodically from FuncAnimation
def animate(i, xs, ys):

    #Aquire and parse data from serial port
    line=ser.readline()      #ascii
    line_as_list = line.split(b',')
    i = int(line_as_list[0])
    relProb = line_as_list[1]
    relProb_as_list = relProb.split(b'\n')
    relProb_float = float(relProb_as_list[0])
	
	# Add x and y to lists
    xs.append(i)
    ys.append(relProb_float)
    rs.append(0.5)

    # Limit x and y lists to 20 items
    #xs = xs[-20:]
    #ys = ys[-20:]

    # Draw x and y lists
    ax.clear()
    ax.plot(xs, ys, label="Experimental Probability")
    ax.plot(xs, rs, label="Theoretical Probability")

    # Format plot
    plt.xticks(rotation=45, ha='right')
    plt.subplots_adjust(bottom=0.30)
    plt.title('This is how I roll...')
    plt.ylabel('Relative frequency')
    plt.legend()
    plt.axis([1, None, 0, 1.1]) #Use for arbitrary number of trials
    #plt.axis([1, 100, 0, 1.1]) #Use for 100 trial demo

# Set up plot to call animate() function periodically
ani = animation.FuncAnimation(fig, animate, fargs=(xs, ys), interval=1000)
plt.show()

In [ ]:
import random
from itertools import count
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

plt.style.use('fivethirtyeight')

x_vals = []
y_vals = []

index = count()


def animate(i):
    data = pd.read_csv('data.csv')
    x = data['x_value']
    y1 = data['total_1']
    y2 = data['total_2']

    plt.cla()

    plt.plot(x, y1, label='Channel 1')
    plt.plot(x, y2, label='Channel 2')

    plt.legend(loc='upper left')
    plt.tight_layout()


ani = FuncAnimation(plt.gcf(), animate, interval=1000)

plt.tight_layout()
plt.show()